In [1]:
'''
Test R1 distillated llama 8b for ner
'''

'\nTest R1 distillated llama 8b for ner\n'

In [1]:
# for modernbetr, amndatory
# !pip install "git+https://github.com/huggingface/transformers.git@6e0515e99c39444caae39472ee1b2fd76ece32f1" --upgrade
# !pip install flash-attn==2.7.2.post1

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
import random
from textblob import TextBlob

In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# read data

In [4]:
complain = "I placed order from his telegram carding account and I pay him 6000₹ he sand me a tracking link but it was not working and I got nothing my order never arrived so I am standing proof of my payment so you can check it"

# sentiemnt

In [5]:
sentiments = []

### find it its a complain using base modernbert

In [12]:
#TODO
model_name = "clapAI/modernBERT-base-multilingual-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(device)

device(type='cuda')

In [13]:
inputs = tokenizer(complain, return_tensors="pt").to(device)
outputs = model(**inputs)

RuntimeError: FlashAttention only supports Ampere GPUs or newer.

### find it its a complain using large modernbert

In [ ]:
#TODO

### find it its a complain using text blob

In [ ]:
sentiments.append(TextBlob(complain).sentiment.polarity)

In [ ]:
sentiments

[0.0]

# if it is a complain

In [14]:
prompt = """The following entities are critical to understand in a life or death situation
1. What is being complained about?
2. Whats the issue?
3. Is it going on for a long time?
4.
The above mentioend needs to be extracted from the sentence and shown in a json format

<SENTENCE>
{}
</SENTENCE>
""".format(complain)
messages = [{"role": "You are in a dire situation and need to answer the questions asked related to the sentence passed", "content": ""},{"role": "user", "content": prompt}]


In [16]:
# r1 llmama 8b
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
model = AutoModelForCausalLM.from_pretrained( model_name, torch_dtype="auto", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.06k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

In [17]:
%%time
text = tokenizer.apply_chat_template(messages, tokenize=False,add_generation_prompt=True)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
generated_ids = model.generate(**model_inputs, temperature=0.1, max_new_tokens=512)
generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<think>
Okay, so I need to figure out what the user is asking for here. They provided a sentence and a JSON output, and they want me to extract certain entities from the sentence. The entities they mentioned are "What is being complained about?", "What is the issue?", and "Is it going on for a long time?".

Looking at the sentence: "I placed an order from his Telegram carding account and I paid him 6000₹ he sent me a tracking link but it was not working and I got nothing my order never arrived so I am standing proof of my payment so you can check it."

First, I need to identify what the complaint is about. The user placed an order and paid money, but didn't receive anything. So the complaint is about not receiving the order after payment.

Next, the issue is that the tracking link wasn't working, and the order never arrived. So the problem is the tracking link not working and the order not arriving.

Lastly, the user mentions they have proof of payment, so it's not going on for a long 